In [ ]:
# Download config and checkpoint using mim
!mim download mmdet --config solov2_r50_fpn_1x_coco --dest .

In [ ]:
!ls
%cd ../../..
!ls

In [ ]:
# Fix all compatibility issues
!pip uninstall numpy scikit-learn scipy mmcv -y
!pip install --no-cache-dir --force-reinstall "numpy<2.0" scipy scikit-learn

# Get PyTorch version and install compatible mmcv
import torch
torch_version = torch.__version__
print(f"PyTorch version: {torch_version}")

# Reinstall mmcv from source
!pip install --no-cache-dir "mmcv>=2.0.0,<2.2.0" --no-build-isolation

In [ ]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'characters-and-dialouges-association-in-comics'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

In [ ]:
import os
import json
import torch
import cv2
import numpy as np
from tqdm import tqdm
from mmdet.apis import init_detector, inference_detector

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")
print(f"Using device: {DEVICE}")


In [ ]:
NOTEBOOK_DIR = os.path.join(BASE_DIR, 'code', 'bubble-detection', 'SOLOv2')
print(f"NOTEBOOK_DIR: {NOTEBOOK_DIR}")

CONFIG_FILE = os.path.join(NOTEBOOK_DIR, 'solov2_r50_fpn_1x_coco.py')
CHECKPOINT_FILE = os.path.join(NOTEBOOK_DIR, 'solov2_r50_fpn_1x_coco_20220512_125858-a357fa23.pth')

# Thư mục chứa ảnh và annotation
MANGA_ROOT = os.path.join(BASE_DIR,'data','Manga109_released_2023_12_07','images')
ANNOTATION_ROOT = os.path.join(BASE_DIR,'data','MangaSegmentation','jsons')

# Thư mục lưu kết quả
OUTPUT_DIR = os.path.join(BASE_DIR,'output','solo_bubble_results')
os.makedirs(OUTPUT_DIR, exist_ok=True)

BUBBLE_ID = 5
SCORE_THR = 0.5

In [ ]:
print("🔹 Loading SOLOv2 model...")
model = init_detector(CONFIG_FILE, CHECKPOINT_FILE, device=DEVICE)
print("✅ Model loaded!")

In [ ]:
json_files = sorted([f for f in os.listdir(ANNOTATION_ROOT) if f.endswith('.json')])
print(f"🔹 Found {len(json_files)} manga titles.")

for json_file in tqdm(json_files, desc="Processing manga titles"):
    json_path = os.path.join(ANNOTATION_ROOT, json_file)
    manga_name = os.path.splitext(json_file)[0]
    image_dir = os.path.join(MANGA_ROOT, manga_name)

    if not os.path.exists(image_dir):
        print(f"⚠️ Skipping {manga_name} (no image folder found).")
        continue

    # Load annotation
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Lọc các annotation có category_id = 5 (bubble)
    bubble_annotations = [ann for ann in data["annotations"] if ann["category_id"] == BUBBLE_ID]
    image_map = {img["id"]: img["file_name"] for img in data["images"]}

    # Tạo folder output riêng cho từng manga
    manga_output = os.path.join(OUTPUT_DIR, manga_name)
    os.makedirs(manga_output, exist_ok=True)

    # -------------------------------
    # 4. Inference từng ảnh có bubble
    # -------------------------------
    seen_images = set()

    for ann in bubble_annotations:
        img_file = image_map.get(ann["image_id"])
        if img_file in seen_images:
            continue
        seen_images.add(img_file)

        img_path = os.path.join(image_dir, img_file)
        if not os.path.exists(img_path):
            continue

        # Chạy SOLOv2 inference
        result = inference_detector(model, img_path)

        # Xuất kết quả ảnh có mask overlay
        save_path = os.path.join(manga_output, os.path.basename(img_path))
        try:
            model.show_result(img_path, result, out_file=save_path, score_thr=SCORE_THR)
        except Exception as e:
            print(f"⚠️ Error on {img_file}: {e}")

print(f"🎉 Inference done! All results saved in: {OUTPUT_DIR}")